GPUで学習を行いましょう

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# 作業ディレクトリを選択 (環境に合わせて変更しましょう)
%cd /content/drive/MyDrive/waveform-classification-sample

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import matplotlib.pyplot as plt

In [ ]:
# 機械学習に使用するデータセットを読み込み
df = pd.read_csv("data.csv")

# 答えとなるデータが answer カラムにある場合は、Xから answer カラムを削除
X = df.drop("answer", axis=1).values
# Yはanswerの内容をone-hotエンコーディングを行って代入
y = to_categorical(df[["plastic_type"]].values)

# テストデータ2割、学習データ8割に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# 学習データの数、テストデータの数、クラス数 (カテゴリ数) を表示
print("train data num:", X_train.shape[0])
print("test data num:", y_test.shape[0])
print("class num:", y_train.shape[1])

In [ ]:
# 入力データの長さ
input_shape = (1, 256)
# クラス数 (カテゴリ数)
class_num = y_train.shape[1]

In [ ]:
# モデルを構築
# 上から順に処理が行われていく
model = Sequential([
    LSTM(512, input_shape=input_shape, return_sequences=True, name="lstm_1"),
    LSTM(256, name="lstm_2"),
    Dense(class_num, activation="softmax", name="dense_1")
])

# モデルのコンパイル
model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

In [ ]:
# 検証データ3割で学習を行う
hist = model.fit(
    X_train, y_train,
    batch_size=128,
    epochs=27,
    validation_split=0.3,
    verbose=1
)

In [ ]:
plt.figure()
plt.title("Accuracy")
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.plot(hist.history["accuracy"], label="train")
plt.plot(hist.history["val_accuracy"], label="val")
plt.legend()

In [ ]:
plt.figure()
plt.title("categorical_crossentropy Loss")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.plot(hist.history["loss"], label="train")
plt.plot(hist.history["val_loss"], label="val")
plt.legend()
plt.show()

In [ ]:
# テストデータと学習データの正解率(accuracy)と損失(loss)を表示
# 一般的に言われる「精度」とは、テストデータの正解率のことを指す
train_score = model.evaluate(X_train, y_train)
test_score = model.evaluate(X_test, y_test)
print("Train loss:", train_score[0])
print("Train accuracy:", train_score[1])
print("Test loss:", test_score[0])
print("Test accuracy:", test_score[1])

In [ ]:
# モデルを表示
model.save("classification-model.h5")